In [1]:
# for vector embedding we need the following libraries- langchain sentence_transformers psycopg2
%pip install langchain_text_splitters 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
#%pip install "numpy<2"
#import numpy as np

In [3]:
%pip install sentence_transformers 

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install psycopg2-binary 


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

/Users/kowsalyasitharthaun/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
#converting the whole data into single vector embeding doesnt make sense coz if we do similarity search asking specific info
# example we ask history of ML, only certain parts of the text data will have related info other parts will have info not related to the queston
# we chunk coz while we do similarity search , it can basically match the information and get parts of info that are similar to the question

text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=400, # it takes 100 characters for each sentence
    chunk_overlap=20, #if text, len = 333, 1st chunk 1-100, 2nd chunk 80-180, chunk3 160-260....so on
    length_function=len,
    is_separator_regex=False,
)

#load the text file
with open ('Machine_Learning.txt', mode = 'r', encoding = 'utf-8') as file:
    text = file.read()
text

'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.[2]\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.\nStatistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\nFrom a theoretical viewpoint, probably approximately correct (PAC) learning provides a framework for describing machine learning.\nThe term mach

In [7]:
#convert text to chunks

texts = text_splitter.create_documents([text])

len(texts)





194

In [8]:
texts

[Document(metadata={}, page_content='Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.[2]'),
 Document(metadata={}, page_content='ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.'),
 Document(metadata={}, page_content='Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\nFrom a theoretical viewpoint, 

In [9]:
texts[0] #print the index 

Document(metadata={}, page_content='Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.[2]')

In [10]:
#chunks
texts[0].page_content # only the text content


'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.[2]'

In [11]:
# create connection to database

import psycopg2

conn = psycopg2.connect(
    dbname = "first_database",
    user ="postgres",
    password = "password",
    host= "localhost",
    port = '5432'
)

cursor = conn.cursor()




In [12]:
#refer for  codes from (sentence transformer doc - in browser search) 
from sentence_transformers import SentenceTransformer

# 1. Load a pretrained Sentence Transformer model
model = SentenceTransformer("all-MiniLM-L6-v2")

/Users/kowsalyasitharthaun/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
#vector embeded values
model.encode(texts[0].page_content)

array([-2.55535729e-02, -4.29134443e-02,  5.89330196e-02,  3.78109189e-03,
       -3.37011740e-02, -2.99725458e-02, -2.72163823e-02, -8.16270411e-02,
       -4.32276092e-02, -6.30126074e-02, -3.72133367e-02, -3.35483532e-03,
       -8.80831480e-03, -8.53131786e-02, -4.96608391e-02,  5.31547405e-02,
        3.54450271e-02,  3.77328247e-02, -1.15392148e-01, -1.02323644e-01,
        2.49585602e-02,  2.22636107e-02, -5.91814816e-02,  3.63597795e-02,
        2.11299397e-02,  4.04828638e-02,  8.26128721e-02,  2.18483470e-02,
        7.05688214e-03, -1.77156180e-02,  4.20297906e-02, -6.41098544e-02,
        2.85089221e-02,  6.27853945e-02, -4.93801571e-02,  3.77191268e-02,
       -7.94791356e-02,  1.75946206e-02,  3.55904177e-02, -2.46199276e-02,
       -3.81329581e-02, -7.41010979e-02,  3.01135220e-02, -3.44706364e-02,
        7.93294385e-02,  1.12843692e-01, -1.58747602e-02, -9.48187113e-02,
       -2.54861172e-02,  3.63724120e-02, -7.75984302e-02, -1.66887846e-02,
       -4.40610573e-02,  

In [14]:
#passing the chunk to the model
x = model.encode(texts[0].page_content)
len(x) # gives the dimentions of that array(384),vector embeddings
# thats only for the text at [0] index (like sample)
#we have to do for all the text in the array


384

In [15]:
cursor.execute("""
    CREATE EXTENSION IF NOT EXISTS vector;
               CREATE TABLE IF NOT EXISTS M_learn_embedding(
               id SERIAL PRIMARY KEY,
               text_chunk TEXT NOT NULL,
               embedding VECTOR(384) NOT NULL
               )
""")
cursor.execute('ROLLBACK')

In [16]:
texts

[Document(metadata={}, page_content='Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Advances in the field of deep learning have allowed neural networks to surpass many previous approaches in performance.[2]'),
 Document(metadata={}, page_content='ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[3][4] The application of ML to business problems is known as predictive analytics.'),
 Document(metadata={}, page_content='Statistics and mathematical optimization (mathematical programming) methods comprise the foundations of machine learning. Data mining is a related field of study, focusing on exploratory data analysis (EDA) via unsupervised learning.[6][7]\nFrom a theoretical viewpoint, 

In [17]:
for eachText in texts:
    txt = eachText.page_content
    embed = model.encode(txt).tolist() #converting the vector embedings into list
    cursor.execute("""
        INSERT INTO M_learn_embedding (text_chunk,embedding) 
        VALUES (%s , %s)
""",(txt,embed))
    
    #text chunk is the column name and embedding contains embedding data

In [18]:
query = "how was machine learning evolved?"
cursor.execute('ROLLBACK')

embed_query =model.encode(query)

In [19]:
print(embed_query)

[-3.86542007e-02 -2.55768783e-02  2.23147832e-02  4.82983366e-02
  3.90278697e-02  1.51656778e-03 -7.11163655e-02 -7.15847220e-03
 -3.12739834e-02  2.95937974e-02 -1.32139353e-02  2.18801461e-02
  4.17647697e-02 -4.34927680e-02 -5.78019284e-02 -2.35071965e-02
 -8.30442309e-02  8.57597124e-03 -3.18168961e-02 -7.86489844e-02
  9.78665892e-03  2.97477320e-02  1.00367246e-02 -2.41257083e-02
  3.36651430e-02  5.64585589e-02  1.90939531e-02  1.26050049e-02
  4.29580659e-02 -2.62806024e-02  6.37034103e-02 -3.49636376e-02
  1.93802658e-02 -9.63917654e-03 -5.25030047e-02 -6.44867262e-03
  1.30610541e-02  1.83598530e-02  1.07301986e-02 -1.79862957e-02
 -5.89869432e-02 -4.17023674e-02  1.09762354e-02 -3.56680751e-02
  9.28014740e-02  5.31581976e-02 -1.66040156e-02 -5.62127791e-02
 -2.22752821e-02 -2.96370126e-03 -1.41422480e-01 -4.98923101e-02
  1.06653981e-02 -3.97839434e-02 -4.92388420e-02  5.16715087e-02
  9.11187846e-03  6.33241907e-02  6.41310364e-02  7.11408863e-03
 -4.47771102e-02 -1.49452

In [20]:
embedList = embed_query.tolist()
cursor.execute(f"SELECT text_chunk from M_Learn_embedding ORDER BY embedding <-> '{embedList}' Limit 6")

In [21]:
results = cursor.fetchall()

In [22]:
print(results)

[('As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were',), ('As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were',), ('As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in hav

In [23]:
for i in results:
    print(i)
    print('\n \n')

('As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were',)

 

('As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested in having machines learn from data. They attempted to approach the problem with various symbolic methods, as well as what were then termed "neural networks"; these were mostly perceptrons and other models that were',)

 

('As a scientific endeavor, machine learning grew out of the quest for artificial intelligence (AI). In the early days of AI as an academic discipline, some researchers were interested i